# Chicken Caesar Salad

**Đề:** (quên lưu)

**FLAG:**: 
>ictf{wHen_dID_cAEseR_cIphERs_gEt_sO_hARd}

# Flip_Flop

**Đề:**

In [ ]:
#!/usr/local/bin/python
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import binascii
import os

print('''
                                        ,,~~~~~~,,..
                             ...., ,'~             |
                             \    V                /
                              \  /                 /
                              ;####>     @@@@@     )
                              ##;,      @@@@@@@    )
                           .##/  ~>      @@@@@   .   .
                          ###''#>              '      '
      .:::::::.      ..###/ #>               '         '
     //////))))----~~ ## #}                '            '
   ///////))))))                          '             '
  ///////)))))))\                        '              '
 //////)))))))))))                                      '
 |////)))))))))))))____________________________________).
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||

(yeah they're not flip flops but close enough)

''')

key = os.urandom(16)
iv = os.urandom(16)
flag = open("flag.txt").read().strip()


for _ in range(3):
	print("Send me a string that when decrypted contains 'gimmeflag'.")
	print("1. Encrypt")
	print("2. Check")
	choice = input("> ")
	if choice == "1":
		cipher = AES.new(key, AES.MODE_CBC, iv)
		pt = binascii.unhexlify(input("Enter your plaintext (in hex): "))
		if b"gimmeflag" in pt:
			print("I'm not making it *that* easy for you :kekw:")
		else:
			print(binascii.hexlify(cipher.encrypt(pad(pt, 16))).decode())
	else:
		cipher = AES.new(key, AES.MODE_CBC, iv)
		ct = binascii.unhexlify(input("Enter ciphertext (in hex): "))
		assert len(ct) % 16 == 0
		if b"gimmeflag" in cipher.decrypt(ct):
			print(flag)
		else:
			print("Bad")

print("Out of operations!")


Solve:

Về cơ bản, đề bắt mình nhập 1 đoạn cipher sao cho sau khi decrypt chứa "gimmieflag" để tìm ra flag và cho mình 3 lần thử: hoặc là encrypt, hoặc là decrypt.

Rõ ràng trong 3 lần thử này, ta có iv và key như nhau, nhưng khi encrypt chữ "gimmieflag" thì tất nhiên nó không cho, và bài cũng không dễ ăn vậy

*Ý tưởng:* 

Bây giờ mình chia phần mình cần nhập (gimmieflag) thành 2 phần. Nhưng sau khi ghép lại, nó vẫn chứa "gimmieflag". Do mình có 3 lần thử

Tạt về lý thuyết, ta có mô hình AES_CBC như sau:

- $C1 = E(P1 xor IV)$
- $C2 = E(P2 xor C1)$

...

**Tiếp cận:**

Gọi C'1 và C'2 là 2 đoạn cipher để mình ghép lại sao cho ra 1 dãy chứa "gimmieflag"

Nếu ta chia phần cần nhập thành 2 dãy: "aaa..gimmie" và "flagaa..aa" sao cho mỗi phần có 16 bit thì sao, để sau mình ghép lại ấy?

Sau khi nhập, ta được:

- $C1 = E(P1 xor IV)$
- $C2 = E(P2 xor IV)$

=> Vậy đâu có đúng?

Chuyển hướng tiếp cận:

Phần đầu là : "aa..gimmieaa..aa" (32bytes), sau khi nhập lần đầu, ta được:

- $C1 = E(P1 xor IV)$
- $C2 = E(P2 xor C1)$

(vì mỗi block chiếm 16 bytes)

Vì block đầu tiên luôn đúng nên mình không cần bận tâm mấy, việc mình cần là làm sao tìmd dược block 2 sao cho $C2 = E(P2 xor C1)$ ($C2 = E(flagaa..aa xor E(aa..aagimmie xor IV))$) (là phần flagaa..aa)

Vậy sau khi nhập lần đầu như trên, ta có được C'1 rồi, giờ tìm C'2, nhưng cái mình cần là phần mình nhập sẽ như thế nào để ra được C'2?

Nếu mình mở rộng độ dài của đoạn lần đầu nhập (thêm 16bytes), đồng nghĩa với việc: $C3 = E(P3 xor C2)$. Nhưng cái mình cần tìm là $P'3 xor C'1$, song, ta có phương trình:

$P3 xor C2 = P'3 xor C'1$

Mà C'1 = C1

=> $P'3 = P3 xor C2 xor C1$




In [ ]:
from pwn import *

s = remote('chal.imaginaryctf.org', 42011)

s.recvuntil('> ')
s.sendline(b'1')

s.recvuntil('Enter your plaintext (in hex): ')
payload1 = 'aaaaaaaagimmefla' + 'aaaaaaaaaaaaaaaa'
s.sendline(bytes(payload1.encode('utf-8').hex(), 'utf-8'))

d = s.recvline().strip().decode('utf-8')[:64]

a = d[:32]
b = d[32:]

payload2 = int('gaaaaaaaaaaaaaaa'.encode('utf-8').hex(), 16)
payload2 ^= (int(a, 16) ^ int(b, 16))

payload2 = hex(payload2)

s.recvuntil('> ')
s.sendline(b'1')
s.recvuntil('Enter your plaintext (in hex): ')

s.sendline(bytes(payload1.encode('utf-8').hex() + payload2[2:], 'utf-8'))

e = s.recvline().strip().decode('utf-8')
e = e[64:96]

s.recvuntil('> ')
s.sendline(b'2')
s.recvuntil('Enter ciphertext (in hex): ')

s.sendline(bytes(a + e, 'utf-8'))

flag = s.recvline()
print(flag)




# Rock_Solid_Algorithm

**Đề:**

n= 18718668654839418101060350414897678724581774081742102287908765212690862231899547405582997157020093499506177632395430572542600019258424947803591395926472246347413986531437177801754324606200243710836609694453888894668656807471052095014376204102474311740080044776201105722801365112971807912406879483156845216746137339614577267869908065296042390812575960639865867729920434603853708907147465162697098688239587320232595412227310236678367

e = 5

c = 4061448515799106340420739691775850071489215699577921072934942485890519294380069123037340174441242842518682390853378784679825023237216051766738593812159344136064529711265570171627670665806072255545198689928996413238102114126558579154343844959868438278433954975590137693439216155482228025380904377837299357044104373966173149290333194304831238889245126840666444234215617022142380016275718234640045049962318290976661640301222078289152


**Solve:**

Tuyệt vời, ta thấy e nhỏ, nên có thể brute_force do $m^e = c + i*n$ <=> $c = m^e mod n$

In [ ]:
from Crypto.Util.number import *
import gmpy2

n = 18718668654839418101060350414897678724581774081742102287908765212690862231899547405582997157020093499506177632395430572542600019258424947803591395926472246347413986531437177801754324606200243710836609694453888894668656807471052095014376204102474311740080044776201105722801365112971807912406879483156845216746137339614577267869908065296042390812575960639865867729920434603853708907147465162697098688239587320232595412227310236678367
e = 5
c = 4061448515799106340420739691775850071489215699577921072934942485890519294380069123037340174441242842518682390853378784679825023237216051766738593812159344136064529711265570171627670665806072255545198689928996413238102114126558579154343844959868438278433954975590137693439216155482228025380904377837299357044104373966173149290333194304831238889245126840666444234215617022142380016275718234640045049962318290976661640301222078289152

for i in range(100):
    temp_c, checked = gmpy2.iroot(c + i*n, e)
    if checked:
        print(i)
        print(long_to_bytes(temp_c))

>FLAG: ictf{wHen_dID_cAEseR_cIphERs_gEt_sO_hARd}